# OBS! Is now a python script

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
plt.style.use('ggplot')

# Args

In [3]:
EXP = "exp3"
PLATFORM = "IONTORRENT"#"ILLUMINA"

In [4]:
MAPPING = "BLAST" #'KMA' # BLAST
BARCODE_SYSTEM = "AKB" #'10x' #'AKB' #

## Input data

In [5]:
tcr_contig_annotations = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_TCR/augmented/tcr.clean.augmented.csv"

In [6]:
if MAPPING == 'KMA':
    barcodes = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_MHC_" + PLATFORM + "/mapping/kma-1t1/output/mapping.clean." + BARCODE_SYSTEM + ".augmented.gz"
if MAPPING == 'BLAST':
    barcodes = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_MHC_" + PLATFORM + "/mapping/blast/blast.annotated.clean." + BARCODE_SYSTEM + ".augmented.tsv"

## Output data

In [7]:
output = "/Volumes/tuba/herpov/tcr-pmhc-sc-project/data/" + EXP + "_CAT_" + PLATFORM + "_" + MAPPING + "_" + BARCODE_SYSTEM + "/tables/tcr_barcode.xlsx"

# Import input

In [8]:
tcr_chain_match_df = pd.read_csv(tcr_contig_annotations)

In [9]:
barcode_specificity_df = pd.read_csv(barcodes, sep="\t") #, usecols=['query_id', 'template_id', 'gem', 'bit_score', 'alignment_length', 'tso', 'b_primer', 'anneal', 'a_primer', 'match'], sep=" ", names=["read_id", "gem", "tso", "b_primer", "anneal", "cd8_primer", "mhc_primer"]

## Merge clonotypes and barcodes

In [10]:
clonotype_barcode_specificity_df = pd.merge(tcr_chain_match_df[['gem', 'clonotype',
                                                                'cdr3_TRA', 'cdr3_nt_TRA',
                                                                'cdr3_TRB', 'cdr3_nt_TRB',
                                                                'umis_TRA', 'umis_diff_TRA', 'single_TRA', 'umis_lst_TRA', 'cdr3_lst_TRA',
                                                                'umis_TRB', 'umis_diff_TRB', 'single_TRB', 'umis_lst_TRB', 'cdr3_lst_TRB']],
                                            barcode_specificity_df,
                                            how='right', on='gem')

column_names_to_be_removed = clonotype_barcode_specificity_df.columns.to_list()
column_names_to_be_removed.remove('umis_lst_TRA')
column_names_to_be_removed.remove('umis_lst_TRB')
column_names_to_be_removed.remove('read_counts_lst_mhc')
column_names_to_be_removed.remove('read_counts_lst_cd8')
column_names_to_be_removed.remove('template_lst_mhc')
column_names_to_be_removed.remove('template_lst_cd8')
column_names_to_be_removed.remove('cdr3_lst_TRA')
column_names_to_be_removed.remove('cdr3_lst_TRB')
column_names_to_be_removed.remove('cdr3_TRA')
column_names_to_be_removed.remove('cdr3_TRB')
column_names_to_be_removed.remove('cdr3_nt_TRA')
column_names_to_be_removed.remove('cdr3_nt_TRB')
column_names_to_be_removed.remove('epitope_lst')

clonotype_barcode_specificity_df.drop_duplicates(subset=column_names_to_be_removed, inplace=True)

In [11]:
print(clonotype_barcode_specificity_df.shape)
print(clonotype_barcode_specificity_df.gem.unique().shape)

(6385, 37)
(6385,)


At this point the table contains clonotype

## Write to excel

In [13]:
clonotype_barcode_specificity_df.to_excel(output, index=False)